In [7]:
#coding=utf-8

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import pandas as pd
import numpy as np
import signal
import os
import json
import sys

In [6]:
#!pip install C:/Users/Liksakov/Downloads/xgboost-0.7-cp36-cp36m-win_amd64.whl

In [10]:
# laod HR data:
dat = pd.read_csv('data/HR.csv')

In [11]:
# trandform data to DMatrix format:
dat_tr = xgb.DMatrix(data=dat.drop(['left'], axis=1).values, label=dat['left'].values)

In [12]:
# Load data directly to DMatrix format:
dat_xgb = xgb.DMatrix('data/HR.csv?format=csv&label_column=6')

Find best params for xgboost model which maximize accuracy on 3-fold cv 

In [13]:
# set params
xgb_params = {
              'max_depth':8,
              'n_estiamtors':10000000,
              'learning_rate':0.001,
              'gamma' :0,
              #'early_stopping_rounds':10000000,
              'min_child_weight':3,
              'subsample':0.8,
              'colsample_bytree':1,
              'reg_alpha':0,
              'reg_gamma':0,
              'scale_pos_weight':1,
              'silent':0,
              'objective':'binary:logistic',
              #'ntread':4,
              'eval_metric':'merror', # misclassification error
              'seed':42
             }

In [16]:
my_xgb_estimator=xgb.XGBClassifier(max_depth=10, n_estimators=1000)
score = np.mean(
                cross_val_score(estimator=my_xgb_estimator, 
                               X=dat.drop(['left'],axis=1).values,
                               y=dat['left'].values,
                               scoring='accuracy', cv=3))
print('{:.4f}'.format(score))

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7212


C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [57]:
my_xgb_estimator=xgb.XGBClassifier(max_depth=8, 
                                   n_estimators=3000,
                                   learning_rate=0.005,
                                   gamma=0,
                                   min_child_weight=6,
                                   col_sample_bytree=0.5,
                                   subsample=1,
                                   reg_alpha=1,
                                   #reg_lambda=1000,
                                   objective='binary:logistic',
                                   seed=42
                                   )
score = np.mean(
                cross_val_score(estimator=my_xgb_estimator, 
                               X=dat.drop(['left'],axis=1).values,
                               y=dat['left'].values,
                               scoring='accuracy', cv=3))
print('{:.4f}'.format(score))

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7785


C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
0.7786 # 5000 + 0.005 + 3
0.7818 # 800 + 0.005 + 7 + min_c_w = 4
0.7827 # 850 +0.095 + 8 + min_c_w = 6
0.7827 # 900 +0.095 + 8 + min_c_w = 6 + reg_alpha = 0.01

0.7826

In [31]:
# use grid search
param_grid = {'max_depth': [5, 8, 12],
              'n_estimators': [800, 1500, 2500],
              'learning_rate': [0.05, 0.01, 0.005, 0.0001],
              'min_child_weight':[4,6],
              'subsample':[0.7, 1.0],
              #'gamma':[0,10],
              'reg_lambda':[0,1000],
              'reg_alpha':[0.01, 0,100]
              }

In [32]:
model_xgb = GridSearchCV(estimator=xgb.XGBClassifier(),
                        param_grid=param_grid,
                        scoring='accuracy', cv=3, verbose=True)

In [33]:
%%time
model_xgb.fit(X=dat.drop(['left'],axis=1).values,
              y=dat['left'].values)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

...

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Liksakov\AppData\Local\Continuum\anaconda3_2\lib\site-packages\sklearn\pr

Wall time: 7h 19min 59s


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 8, 12], 'n_estimators': [800, 1500, 2500], 'learning_rate': [0.05, 0.01, 0.005, 0.0001], 'min_child_weight': [4, 6], 'subsample': [0.7, 1.0], 'reg_lambda': [0, 1000], 'reg_alpha': [0.01, 0, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=True)

In [34]:
model_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=8, min_child_weight=4, missing=None, n_estimators=2500,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.01, reg_lambda=1000, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

In [35]:
#XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
#       max_depth=8, min_child_weight=4, missing=None, n_estimators=2500,
#       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
#      reg_alpha=0.01, reg_lambda=1000, scale_pos_weight=1, seed=None,
#       silent=True, subsample=1.0)

In [16]:
SCRIPT_DIR = 'C:/Users/Kirill/Desktop/data science/dmia'
path = SCRIPT_DIR +'/xgboost_params_example.json'
with open(path, 'r') as f:
    params = json.load(f)

params

{'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 1000,
 'seed': 42}

In [23]:
SCRIPT_DIR = 'C:/Users/Kirill/Desktop/data science/dmia'


def signal_handler(signum, frame):
    raise Exception("Timed out!")

In [166]:
class Checker(object):
    def __init__(self, data_path=SCRIPT_DIR + '/data/HR.csv'):
        df = pd.read_csv(data_path)
        target = 'left'
        features = [c for c in df if c != target]
        self.target = np.array(df[target])
        self.data = np.array(df[features])

    def check(self, params_path):
       # try:
            with open(params_path, 'r') as f:
                params = json.load(f)
                #signal.signal(signal.SIGALRM, signal_handler)
                # Time limit на эту задачу 2 минуты
                #signal.alarm(120)
                estimator = xgb.XGBClassifier(**params)
                score = np.mean(cross_val_score(
                    estimator, self.data, self.target,
                    scoring='accuracy', 
                    cv=3
                ))
       # except:
       #     print("Unexpected error:", sys.exc_info()[0])
       #     score = None
        
            return score

In [167]:
%%time
print( '{:.4f}'.format(Checker().check(SCRIPT_DIR + '/xgboost_params_example.json')))

0.7660
Wall time: 21.1 s
